In [2]:
import pandas as pd

In [ ]:
# セル2: CSVファイルの読み込み（パスは適宜変更）
df = pd.read_csv('../data/metadata_preprocessed.csv')

C:\Users\taka2\AppData\Local\Temp\ipykernel_28740\235589160.py:4: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/cleaned_metadata_output.csv')


### 불필요한 컬럼 및 전처리 필요한 컬럼
- original_url,pri_domain,final_url 불필요해 보임 삭제
- type 는 url_type로 숫자로 가능 -> type 삭제
- url_region로 string이라 더미변수로 변한해서 전처리 필요
- redirect_chain도 당장 할수 있는게 없음 -> 삭제
- head_elemants는 분리를 해서 처리함 -> 삭제
- meta_refrech_url는 has_mata_refresh로 처리해서 필요없음 -> 삭제
- title_test, meta_description,og_title,og_image,link_stylesheets는 있냐 없냐로 true, flase로 처리 
### 시작화로 사용 가능한 값
- url_region는 어느 나라가 type걸로 많은지 시작적으로 표현 가능
- root_domain는 단어로 시각화...?
- title_test, meta_description,og_title,og_image,link_stylesheets는 단어로 분리 하고 시작화 및 영량을 얼마나 주는지 확인

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371357 entries, 0 to 371356
Data columns (total 35 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Unnamed: 0           371357 non-null  int64  
 1   url                  371357 non-null  object 
 2   type                 371357 non-null  object 
 3   original_url         371357 non-null  object 
 4   url_type             371357 non-null  int64  
 5   url_len              371357 non-null  int64  
 6   pri_domain           371357 non-null  object 
 7   letters_count        371357 non-null  int64  
 8   digits_count         371357 non-null  int64  
 9   special_chars_count  371357 non-null  int64  
 10  shortened            371357 non-null  int64  
 11  is_domain_in_url     371357 non-null  int64  
 12  secure_http          371357 non-null  int64  
 13  have_ip              371357 non-null  object 
 14  url_region           371357 non-null  object 
 15  root_domain      

In [3]:
df.head()

,Unnamed: 0,url,type,original_url,url_type,url_len,pri_domain,letters_count,digits_count,special_chars_count,...,meta_count,link_count,script_count,title_count,title_text,meta_description,og_title,og_image,script_srcs,link_stylesheets
0,2,bopsecrets.org/rexroth/cr/1.htm,benign,bopsecrets.org/rexroth/cr/1.htm,0,31,bopsecrets.org,25,1,5,...,7.0,1.0,0.0,1.0,Classics Revisited (1),NaN,NaN,NaN,[],['../../css/rexroth.css']
1,6,espn.go.com/nba/player/_/id/3457/brandon-rush,benign,espn.go.com/nba/player/_/id/3457/brandon-rush,0,45,espn.go.com,31,4,10,...,31.0,62.0,0.0,1.0,Brandon Rush - Maccabi Haifa Forward - ESPN,View the profile of Maccabi Haifa Forward Bran...,Brandon Rush - Maccabi Haifa Forward - ESPN,https://a.espncdn.com/combiner/i?img=/i/headsh...,[],['//cdn1.espn.net/fitt/e875a0841ebc-release-04...
2,7,yourbittorrent.com/?q=anthony-hamilton-soulife,benign,yourbittorrent.com/?q=anthony-hamilton-soulife,0,46,yourbittorrent.com,40,0,6,...,4.0,11.0,0.0,1.0,YourBittorrent,NaN,NaN,NaN,[],"['https://y.t0r.site/static/style4.1.css', 'ht..."
3,9,allmusic.com/album/crazy-from-the-heat-r16990,benign,allmusic.com/album/crazy-from-the-heat-r16990,0,45,allmusic.com,33,5,7,...,30.0,52.0,0.0,1.0,Crazy from the Heat - David Lee Roth | Album |...,Crazy from the Heat by David Lee Roth released...,Crazy from the Heat - David Lee Roth | Album |...,https://fastly-s3.allmusic.com/release/mr00003...,[],['https://fonts.googleapis.com/css2?family=Bev...
4,10,corporationwiki.com/Ohio/Columbus/frank-s-bens...,benign,corporationwiki.com/Ohio/Columbus/frank-s-bens...,0,62,corporationwiki.com,47,7,8,...,27.0,15.0,0.0,1.0,Frank Benson - Director for Clp Holding Corpor...,View Frank S Benson's profile for company asso...,Frank Benson - Director for Clp Holding Corpor...,https://cdn.corporationwiki.com/profiles/graph...,[],['https://cdn.corporationwiki.com/public/css/b...


In [6]:
# 以下のカラムは目的に不要と判断して削除
cols_to_drop = [
    'original_url',        # url と内容重複
    'pri_domain',          # root_domain などと重複性あり
    'final_url',           # redirect後のurlで今回不要
    'type',                # url_type に変換可能
    'redirect_chain',      # 当面利用しない
    'head_elements',       # 分解処理済み
    'meta_refresh_url',    # has_meta_refresh で代替可能
    'script_count'         # クローリング不完全なため除外
]

df = df.drop(columns=cols_to_drop)

### title_text, meta_description, og_title, og_image, link_stylesheets 를 boolen로 변환

In [9]:
# 存在すれば True（=1）、なければ False（=0）
df['has_title'] = df['title_text'].apply(
    lambda x: isinstance(x, str) and x.strip() != ''
)
df['has_meta_description'] = df['meta_description'].notna()
df['has_og_image'] = df['og_image'].notna()
df['has_og_title'] = df['og_title'].notna()
df['has_link_stylesheet'] = df['link_stylesheets'].apply(
    lambda x: isinstance(x, str) and x.strip() != '[]'
)

### 더 이상 필요하지 않은 원시 컬럼 삭제

In [11]:
df = df.drop(columns=['title_text', 'meta_description', 'og_title', 'og_image', 'link_stylesheets'])

In [16]:
df = df.drop(columns=['script_srcs'])

In [18]:
print(df['have_ip'].value_counts())

have_ip
0       195478
0       174749
True      1130
Name: count, dtype: int64


In [19]:
# すべて文字列に変換して "true" という文字列だけを True にする（他は False）
df['have_ip'] = df['have_ip'].astype(str).str.lower().eq('true')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371357 entries, 0 to 371356
Data columns (total 24 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   url                   371357 non-null  object 
 1   url_type              371357 non-null  int64  
 2   url_len               371357 non-null  int64  
 3   letters_count         371357 non-null  int64  
 4   digits_count          371357 non-null  int64  
 5   special_chars_count   371357 non-null  int64  
 6   shortened             371357 non-null  int64  
 7   is_domain_in_url      371357 non-null  int64  
 8   secure_http           371357 non-null  int64  
 9   have_ip               371357 non-null  bool   
 10  url_region            371357 non-null  object 
 11  root_domain           371227 non-null  object 
 12  redirect_count        371357 non-null  int64  
 13  timeout               371357 non-null  bool   
 14  has_meta_refresh      371357 non-null  bool   
 15  

In [21]:
df.to_csv("../data/url_metadata_cleaned.csv", index=False)